In [2]:
import numpy as np 
import pandas as pd 
import tqdm
import string 
from sklearn.svm import LinearSVC 
from sklearn.calibration import CalibratedClassifierCV 

from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 
from sklearn.decomposition import TruncatedSVD
from sklearn import ensemble, metrics, model_selection, naive_bayes

In [3]:
import glob
import pandas as pd
from tensorflow import keras
import numpy as np
import os 
from sklearn.model_selection import StratifiedKFold, KFold
import matplotlib.pylab as plt
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.layers import TimeDistributed, Conv2D, Conv2DTranspose, MaxPooling2D, AveragePooling2D, BatchNormalization, concatenate, Input, ConvLSTM2D, Reshape, Conv3D, Flatten, LSTM, GRU, Dense,Dropout, Add
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Bidirectional, Conv1D, MaxPooling1D, GlobalMaxPooling1D, GlobalMaxPool1D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.models import Sequential, load_model
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
import re 
from catboost import CatBoostClassifier

In [4]:
train = pd.read_csv('./storage/fintech_nlp/lgbm_train_df.csv') 
test = pd.read_csv('./storage/fintech_nlp/lgbm_test_df.csv')

In [7]:
# add feature ord length 
train['small_ord'] = train['ord'] <= 8
test['small_ord'] = test['ord'] <= 8

In [9]:
x_train = train.drop(['n_id','info'], axis = 1) 
y_train = train['info'] 

In [10]:
x_test = test.drop(['n_id','id'], axis=1) 

In [11]:
x_train.shape, y_train.shape, x_test.shape

((118745, 68), (118745,), (142565, 68))

In [12]:
text_cols = ['title','content','title_tokenized','content_tokenized']

In [13]:
k = 10 
models = [] 
kfold = StratifiedKFold(n_splits = k, shuffle = True, random_state = 20201225) 
for n_fold, (train_idx, val_idx) in enumerate(kfold.split(x_train, y_train)): 
    train_x, val_x = x_train.loc[train_idx,:], x_train.loc[val_idx,:]
    train_y, val_y = y_train.loc[train_idx], y_train.loc[val_idx] 
    
    model = CatBoostClassifier(iterations = 10000, 
                               learning_rate = 0.01, 
                               l2_leaf_reg = 3.5,
                               depth = 8, 
                               loss_function = 'Logloss',
                               eval_metric = 'Logloss',
                               use_best_model = True, 
                               task_type = 'GPU',
                               verbose = 200,
                               random_seed = 88888)  
    
    
    model.fit(train_x, train_y, eval_set = (val_x, val_y), text_features = text_cols) 
    models.append(model)

0:	learn: 0.6709483	test: 0.6705058	best: 0.6705058 (0)	total: 27.9ms	remaining: 4m 38s
200:	learn: 0.0463822	test: 0.0428426	best: 0.0428426 (200)	total: 4.87s	remaining: 3m 57s
400:	learn: 0.0341269	test: 0.0315937	best: 0.0315937 (400)	total: 9.81s	remaining: 3m 54s
600:	learn: 0.0290260	test: 0.0272363	best: 0.0272363 (600)	total: 14.7s	remaining: 3m 49s
800:	learn: 0.0261254	test: 0.0249062	best: 0.0249062 (800)	total: 19.3s	remaining: 3m 41s
1000:	learn: 0.0242433	test: 0.0234795	best: 0.0234795 (1000)	total: 23.4s	remaining: 3m 30s
1200:	learn: 0.0228317	test: 0.0224379	best: 0.0224379 (1200)	total: 27.5s	remaining: 3m 21s
1400:	learn: 0.0216792	test: 0.0216028	best: 0.0216028 (1400)	total: 31.6s	remaining: 3m 13s
1600:	learn: 0.0206754	test: 0.0209091	best: 0.0209091 (1600)	total: 35.6s	remaining: 3m 6s
1800:	learn: 0.0197930	test: 0.0203082	best: 0.0203082 (1800)	total: 39.6s	remaining: 3m
2000:	learn: 0.0189666	test: 0.0197726	best: 0.0197726 (2000)	total: 43.6s	remaining: 2m

In [15]:
pred1 = models[0].predict_proba(x_test) 
pred2 = models[1].predict_proba(x_test) 
pred3 = models[2].predict_proba(x_test) 
pred4 = models[3].predict_proba(x_test) 
pred5 = models[4].predict_proba(x_test)
pred6 = models[5].predict_proba(x_test) 
pred7 = models[6].predict_proba(x_test) 
pred8 = models[7].predict_proba(x_test) 
pred9 = models[8].predict_proba(x_test) 
pred10 = models[9].predict_proba(x_test) 

In [11]:
np.save('./storage/fintech_nlp/cat1.npy',pred1) 
np.save('./storage/fintech_nlp/cat2.npy',pred2)
np.save('./storage/fintech_nlp/cat3.npy',pred3)
np.save('./storage/fintech_nlp/cat4.npy',pred4) 
np.save('./storage/fintech_nlp/cat5.npy',pred5) 
np.save('./storage/fintech_nlp/cat6.npy',pred6) 
np.save('./storage/fintech_nlp/cat7.npy',pred7)
np.save('./storage/fintech_nlp/cat8.npy',pred8)
np.save('./storage/fintech_nlp/cat9.npy',pred9)
np.save('./storage/fintech_nlp/cat10.npy',pred10)

In [16]:
pred_avg = (pred1 + pred2 + pred3 + pred4 + pred5 + pred6 + pred7 + pred8 + pred9 + pred10)/10.0 

In [19]:
final_pred = pred_avg[:,1]

In [20]:
class_pred = np.where(final_pred > 0.5, 1, 0).reshape(-1)  
submission = pd.read_csv('./storage/fintech_nlp/sample_submission.csv') 
submission['info'] = class_pred 
submission.to_csv('./storage/fintech_nlp/catboost_10_fold.csv',index=False)

In [21]:
# print submission file to check 
submission 

,id,info
0,NEWS00237_1,0
1,NEWS00237_2,0
2,NEWS00237_3,0
3,NEWS00237_4,0
4,NEWS00237_5,0
...,...,...
142560,NEWS09482_72,1
142561,NEWS09482_73,1
142562,NEWS09482_74,1
142563,NEWS09482_75,1
